# ST-Power-Market-Challenge Analysis
An analysis of the provided trading data in order to get some insights in the Iberian continuous intraday power market.

## Imports

In [20]:
import pandas as pd
import numpy as np
import zipfile
import os

## Inputs

In [21]:
zip_path = 'data/all_trades_2022.zip'
extract_path = 'data/all_trades_2022'
file_name = 'all_2022.csv'

## Load Data

In [22]:
# Extract data if still a zip
if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

data = pd.read_csv(os.path.join(extract_path, file_name))

## 1. Analysis of Top 10 Agents by Traded Volumes

In [23]:
# Calculate total energy traded for buys and sells separately
buy_volume = data.groupby(
    'buy_agent')['energy'].sum().reset_index(name='buy_volume')
sell_volume = data.groupby(
    'sell_agent')['energy'].sum().reset_index(name='sell_volume')

In [24]:
# Merge buy and sell volumes and fill NaN values
agent_volumes = pd.merge(
    buy_volume, sell_volume, left_on='buy_agent', right_on='sell_agent', how='outer')
agent_volumes.fillna(0, inplace=True)

In [25]:
# Calculate total volume (buy + sell) 
agent_volumes['total_volume'] = agent_volumes['buy_volume'] + \
    agent_volumes['sell_volume']

In [26]:
# Combine buy and sell_agent rows to don´t forget about only sellers
agent_volumes['agent'] = np.where(
    agent_volumes['buy_agent'] == 0, agent_volumes['sell_agent'], agent_volumes['buy_agent'])

agent_volumes = agent_volumes[[
    'agent', 'total_volume', 'buy_volume', 'sell_volume']]

In [27]:
# Sort by total volume and select the top 10
top_10_traders = agent_volumes.sort_values(
    by='total_volume', ascending=False).head(10)
top_10_traders

,agent,total_volume,buy_volume,sell_volume
84,IBGES,1327221.0,632072.8,695148.2
70,GESTE,933996.8,380424.2,553572.6
132,SOREX,852699.1,361439.0,491260.1
41,EDPGP,665977.5,412072.6,253904.9
71,GNCO,623237.5,209324.0,413913.5
38,ECYR,522970.3,267801.0,255169.3
82,HISPE,515821.2,242938.7,272882.5
72,GNRA,414352.4,193593.9,220758.5
53,ENDG,349740.8,168698.7,181042.1
80,HC G,301884.0,195132.7,106751.3


## 2. Calculating of an hourly Volume-Weighted Average Price (vwap).

In [28]:
# Calculate monetary value
data['monetary_value'] = data['energy'] * data['price']

In [29]:
# Calculate hourly vwap
hourly_vwap = data.groupby('contract_start').agg(sum_monetary_value=('monetary_value', 'sum'),
                                                 sum_energy=('energy', 'sum'))
hourly_vwap['vwap'] = hourly_vwap['sum_monetary_value'] / \
    hourly_vwap['sum_energy']

In [30]:
# Merge hourly vwap info back into data
data = pd.merge(data, hourly_vwap['vwap'], on='contract_start', how='left')

In [31]:
data.head()

,Unnamed: 0,date,contract,buy_agent,buy_uof,buy_zone,sell_agent,sell_uof,sell_zone,price,energy,timestamp,contract_start,contract_end,monetary_value,vwap
0,0,2022-01-01,20220101 09:00-20220101 10:00,GNRA,GNVD132,10YES-REE------0,NaN,NaN,NaN,69.90,0.1,31/12/2021 22:00:07,2022-01-01 09:00:00,2022-01-01 10:00:00,6.990,75.428597
1,1,2022-01-01,20220101 09:00-20220101 10:00,GNRA,GNVD132,10YES-REE------0,NaN,NaN,NaN,70.00,4.9,31/12/2021 22:00:07,2022-01-01 09:00:00,2022-01-01 10:00:00,343.000,75.428597
2,2,2022-01-01,20220101 09:00-20220101 10:00,GNRA,GNVD132,10YES-REE------0,NaN,NaN,NaN,70.00,2.1,31/12/2021 22:00:07,2022-01-01 09:00:00,2022-01-01 10:00:00,147.000,75.428597
3,3,2022-01-01,20220101 09:00-20220101 10:00,GNRA,GNVD132,10YES-REE------0,NaN,NaN,NaN,70.00,0.1,31/12/2021 22:00:07,2022-01-01 09:00:00,2022-01-01 10:00:00,7.000,75.428597
4,4,2022-01-01,20220101 09:00-20220101 10:00,GNRA,GNVD132,10YES-REE------0,NaN,NaN,NaN,72.96,2.8,31/12/2021 22:00:07,2022-01-01 09:00:00,2022-01-01 10:00:00,204.288,75.428597


## 3. Scoring of agents' trades according to their spread with vwap. 

In [32]:
# Add spread and score columns
data['spread'] = data['price'] - data['vwap']
data['score'] = data['spread'] / data['vwap']
data['weighted_score'] = data['score']*data['energy']

In [33]:
# Calculate total scores for buys and sells separately
buy_score = data.groupby('buy_agent')[
    'weighted_score'].sum().reset_index(name='buy_score')
sell_score = data.groupby('sell_agent')[
    'weighted_score'].sum().reset_index(name='sell_score')

In [34]:
# Merge buy and sell scores and fill NaN values
temp = pd.merge(
    buy_score, sell_score, left_on='buy_agent', right_on='sell_agent', how='outer')
temp.fillna(0, inplace=True)

In [35]:
# Calculate total score. Negative buy and positive sell values favorable
temp['overall_score'] = temp['sell_score'] - temp['buy_score']

In [36]:
# Combine buy and sell_agent rows to don´t forget about only sellers
temp['agent'] = np.where(
    temp['buy_agent'] == 0, temp['sell_agent'], temp['buy_agent'])

In [37]:
# Merge with agent_volumes
agent_metrics = pd.merge(
    temp[['agent', 'buy_score', 'sell_score', 'overall_score']], agent_volumes, on='agent')

In [38]:
# Sort by total volume and select the top 10
top_10_traders = agent_metrics.sort_values(
    by='total_volume', ascending=False).head(10)

top_10_traders

,agent,buy_score,sell_score,overall_score,total_volume,buy_volume,sell_volume
84,IBGES,-14785.614667,-32250.512383,-17464.897716,1327221.0,632072.8,695148.2
70,GESTE,-52409.149375,65536.464874,117945.614249,933996.8,380424.2,553572.6
132,SOREX,-11766.849779,-5590.765606,6176.084173,852699.1,361439.0,491260.1
41,EDPGP,-30383.599104,1425.812331,31809.411435,665977.5,412072.6,253904.9
71,GNCO,-19122.889851,-14857.838608,4265.051243,623237.5,209324.0,413913.5
38,ECYR,-13828.236192,-15225.093674,-1396.857482,522970.3,267801.0,255169.3
82,HISPE,-6516.603350,-4911.215279,1605.388071,515821.2,242938.7,272882.5
72,GNRA,-20032.004278,26781.492049,46813.496327,414352.4,193593.9,220758.5
53,ENDG,-9023.367640,-8052.423620,970.944020,349740.8,168698.7,181042.1
80,HC G,-2915.638239,-9113.807516,-6198.169277,301884.0,195132.7,106751.3
